# Data Science Capstone Project

#### This notebook will be used to contain the contents of my Data Science Capstone Project and the tasks associated with it

In [5]:
import pandas as pd
import numpy as np
import folium

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [7]:
#Import libraries for parsing wikipedia page
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [8]:
#Read the html of the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response)

<Response [200]>


In [9]:
#Create a BeautifulSoup html parsing object
soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
#Extract the table data frame the page as a list
wiki_page = soup.findAll('td')
for script in soup(["script", "style"]):
    script.extract()    # rip it out

text = soup.get_text()
lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk for chunk in chunks if chunk)

html_list = text.split('\n')[13:877]

In [11]:
#create a dataframe and fill it with the table data
nbhd_df = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
p_list = []
b_list = []
n_list = []

for x in range(0,288):
    p_list.append(html_list[x*3])
    b_list.append(html_list[x*3+1])
    n_list.append(html_list[x*3+2])
    
nbhd_df['Postcode'] = p_list
nbhd_df['Borough'] = b_list
nbhd_df['Neighbourhood'] = n_list

In [12]:
#remove not assigned boroughs, not assigned neighbourhoods are set equal to borough name
nbhd_df = nbhd_df.loc[nbhd_df['Borough'] != 'Not assigned']
for x in range(len(nbhd_df)):
    if nbhd_df['Neighbourhood'].iloc[x] == 'Not assigned':
        nbhd_df['Neighbourhood'].iloc[x] = nbhd_df['Borough'].iloc[x]

In [13]:
#group and join neighbourhoods by ', '
grouped_df = nbhd_df.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [14]:
#Shape
grouped_df.shape

(103, 3)

In [15]:
#Read in Lats and Longs and assign to appropriate postcode
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
grouped_df['Latitude'] = ''
grouped_df['Longitude'] = ''
for x in range(len(df_ll)):
    temp_str = df_ll['Postal Code'].iloc[x]
    grouped_df.loc[grouped_df['Postcode'] == temp_str, 'Latitude'] = df_ll['Latitude'].iloc[x]
    grouped_df.loc[grouped_df['Postcode'] == temp_str, 'Longitude'] = df_ll['Longitude'].iloc[x]
    

In [16]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

for lat, lng, borough, neighborhood in zip(grouped_df['Latitude'], grouped_df['Longitude'], grouped_df['Borough'], grouped_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto